# Task Scheduling Project
## Run in Google Colab

This notebook will automatically:
1. Install .NET SDK
2. Clone the repository from GitHub
3. Restore and build the project
4. Run all experiments
5. Display results

**Instructions:**
- Click "Runtime" → "Run all" (or press Ctrl+F9)
- Wait for all cells to complete
- Results will be displayed at the bottom


In [ ]:
# Step 1: Install system dependencies and .NET SDK 8.0
print("=" * 70)
print("Step 1: Installing .NET SDK 8.0")
print("=" * 70)

# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libicu-dev libunwind8 libssl-dev curl

# Download and install .NET SDK 8.0
!wget -q https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!bash dotnet-install.sh --channel 8.0 --install-dir /root/.dotnet --verbose

# Set environment variables
import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')
os.environ['DOTNET_ROOT'] = '/root/.dotnet'

# Verify installation
print("\nVerifying .NET installation...")
result = !export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && dotnet --version 2>&1
if result and len(result) > 0:
    print(f"✅ .NET version: {result[0]}")
else:
    print("❌ .NET installation failed. Trying alternative method...")
    !bash -c 'export PATH=$PATH:/root/.dotnet && dotnet --version'


In [ ]:
# Step 2: Clone repository from GitHub
print("=" * 70)
print("Step 2: Cloning Repository from GitHub")
print("=" * 70)

import os
import shutil

# Start from /content directory to avoid nesting issues
if os.path.exists('/content'):
    os.chdir('/content')
    print(f"Changed to /content directory")

# Remove existing directory if it exists
if os.path.exists('Task-Scheduling'):
    print("Removing existing Task-Scheduling directory...")
    shutil.rmtree('Task-Scheduling')

# Clone repository
print("Cloning repository...")
!git clone https://github.com/MohamadJehad/Task-Scheduling.git

# Change to project directory
if os.path.exists('Task-Scheduling'):
    os.chdir('Task-Scheduling')
    print(f"\n✅ Repository cloned successfully!")
    print(f"📁 Current directory: {os.getcwd()}")
    print(f"📁 Project file exists: {os.path.exists('TaskScheduling.csproj')}")
else:
    print("❌ Error: Task-Scheduling directory not found after clone")


In [ ]:
# Step 3: Restore NuGet packages
print("=" * 70)
print("Step 3: Restoring NuGet Packages")
print("=" * 70)

import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')

# Restore packages
result = !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && cd Task-Scheduling && dotnet restore' 2>&1

if any('error' in line.lower() or 'failed' in line.lower() for line in result):
    print("\n❌ Restore failed. Output:")
    for line in result:
        print(line)
else:
    print("\n✅ Packages restored successfully!")


In [ ]:
# Step 4: Build the project
print("=" * 70)
print("Step 4: Building Project")
print("=" * 70)

import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')

# Build project
result = !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && cd Task-Scheduling && dotnet build --no-restore' 2>&1

if any('error' in line.lower() or 'failed' in line.lower() for line in result):
    print("\n❌ Build failed. Output:")
    for line in result:
        print(line)
else:
    print("\n✅ Project built successfully!")


In [ ]:
# Step 5: Run the project
print("=" * 70)
print("Step 5: Running Task Scheduling Experiments")
print("=" * 70)
print("\nThis may take a few minutes...\n")

import os
import subprocess
import sys

os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')
os.environ['DOTNET_ROOT'] = '/root/.dotnet'

# Ensure we're in the project directory (should already be there from Step 2)
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# If we're not in Task-Scheduling, navigate to it
if os.path.basename(current_dir) != 'Task-Scheduling':
    if os.path.exists(os.path.join(current_dir, 'Task-Scheduling')):
        os.chdir(os.path.join(current_dir, 'Task-Scheduling'))
        print(f"Changed to: {os.getcwd()}")
    else:
        print("⚠️  Warning: Task-Scheduling directory not found in current location")

project_dir = os.getcwd()
print(f"Running from: {project_dir}\n")

# Verify we have the project file
if not os.path.exists(os.path.join(project_dir, 'TaskScheduling.csproj')):
    print(f"❌ Error: TaskScheduling.csproj not found in {project_dir}")
    print("Available files:")
    for item in os.listdir(project_dir)[:10]:  # Show first 10 items
        print(f"  - {item}")
else:
    # Run the project using bash (more reliable in Colab)
    print("Running project...\n")
    !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && export DOTNET_ROOT=/root/.dotnet && dotnet run --no-build'


In [ ]:
# Step 6: Display all generated reports
print("=" * 70)
print("Step 6: Generated Reports")
print("=" * 70)

import os
from IPython.display import display, Markdown

# Find the reports directory - search intelligently
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

reports_dir = None

# Strategy 1: Check current directory and parent directories (walk up)
search_dir = current_dir
for _ in range(5):  # Check up to 5 levels up
    test_reports = os.path.join(search_dir, 'reports')
    if os.path.exists(test_reports) and os.path.isdir(test_reports):
        # Check if it has .txt files (actual reports)
        try:
            files = [f for f in os.listdir(test_reports) if f.endswith('.txt')]
            if files:
                reports_dir = test_reports
                print(f"✅ Found reports directory: {reports_dir}")
                break
        except:
            pass
    # Also check if we're in a Task-Scheduling directory
    if os.path.basename(search_dir) == 'Task-Scheduling':
        test_reports = os.path.join(search_dir, 'reports')
        if os.path.exists(test_reports):
            reports_dir = test_reports
            print(f"✅ Found reports in Task-Scheduling: {reports_dir}")
            break
    search_dir = os.path.dirname(search_dir)
    if search_dir == '/' or search_dir == search_dir:  # Reached root or no change
        break

# Strategy 2: Search from /content directory
if not reports_dir:
    content_dir = '/content'
    if os.path.exists(content_dir):
        for root, dirs, files in os.walk(content_dir):
            if 'reports' in dirs:
                test_reports = os.path.join(root, 'reports')
                try:
                    report_files = [f for f in os.listdir(test_reports) if f.endswith('.txt')]
                    if report_files:
                        reports_dir = test_reports
                        print(f"✅ Found reports directory via search: {reports_dir}")
                        break
                except:
                    pass

# Strategy 3: Search current directory tree
if not reports_dir:
    for root, dirs, files in os.walk('.'):
        if 'reports' in dirs:
            test_reports = os.path.join(root, 'reports')
            try:
                report_files = [f for f in os.listdir(test_reports) if f.endswith('.txt')]
                if report_files:
                    reports_dir = test_reports
                    print(f"✅ Found reports directory in tree: {reports_dir}")
                    break
            except:
                pass

if reports_dir and os.path.exists(reports_dir):
    files = [f for f in os.listdir(reports_dir) if f.endswith('.txt')]
    files.sort()
    
    if files:
        print(f"\n📊 Found {len(files)} report file(s):\n")
        
        # Find Master Summary report
        master_summary = [f for f in files if 'MasterSummary' in f]
        
        if master_summary:
            print(f"\n{'='*70}")
            print(f"📄 MASTER SUMMARY REPORT: {master_summary[0]}")
            print(f"{'='*70}\n")
            
            with open(os.path.join(reports_dir, master_summary[0]), 'r', encoding='utf-8') as f:
                content = f.read()
                # Display as code block for better formatting
                display(Markdown(f"```\n{content}\n```"))
            
            print(f"\n{'='*70}\n")
        
        # List all other reports
        other_reports = [f for f in files if 'MasterSummary' not in f]
        if other_reports:
            print(f"\n📋 Individual Instance Reports ({len(other_reports)} files):")
            for file in other_reports:
                file_path = os.path.join(reports_dir, file)
                file_size = os.path.getsize(file_path)
                print(f"  • {file} ({file_size:,} bytes)")
    else:
        print("\n⚠️  No report files found in reports directory.")
else:
    print(f"\n⚠️  Reports directory not found.")
    print(f"Current directory: {current_dir}")
    print("\nSearching for reports directory...")
    
    # List all Task-Scheduling directories
    task_scheduling_dirs = []
    for root, dirs, files in os.walk('/content'):
        if 'Task-Scheduling' in root:
            task_scheduling_dirs.append(root)
    
    if task_scheduling_dirs:
        print(f"\nFound {len(task_scheduling_dirs)} Task-Scheduling directory(ies):")
        for ts_dir in task_scheduling_dirs[:5]:  # Show first 5
            reports_path = os.path.join(ts_dir, 'reports')
            if os.path.exists(reports_path):
                files_count = len([f for f in os.listdir(reports_path) if f.endswith('.txt')])
                print(f"  📁 {ts_dir}")
                print(f"     Reports: {reports_path} ({files_count} .txt files)")
            else:
                print(f"  📁 {ts_dir} (no reports folder)")
    
    # Also check current directory structure
    print(f"\nCurrent directory structure:")
    try:
        for item in os.listdir('.'):
            item_path = os.path.join('.', item)
            if os.path.isdir(item_path):
                print(f"  📂 {item}/")
                if item == 'reports':
                    print(f"     ⚠️  Found 'reports' but it may be empty or in wrong location")
    except:
        pass

print(f"\n{'='*70}")
print("✅ All done! Reports are saved in the 'reports' folder.")
print(f"{'='*70}")
